# Agent-Based Simulation 

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path
import random 
import pandas as pd
from matplotlib.collections import LineCollection
import matplotlib as mpl
import numpy as np
from scipy import optimize as opt
from scipy import integrate as intg
from scipy.optimize import least_squares
from scipy.stats import beta
from scipy.stats import bernoulli
from scipy.stats import cumfreq, beta
from utils import Agent, kill
%matplotlib inline
plt.style.use('../notebook.mplstyle')

SEED = 1


## Exogenous Parameters


In [ ]:
# Setting exogenous parameters
def reset_exog_params():
    global Bm, Bw, bm_vals, bw_vals, δ, um, uw, Fm, Fw, λm, λw, T  
    Bm = 10
    Bw = 10
    bm_vals = range(1,Bm+1) 
    bw_vals = range(1,Bw+1)
    
    δ = 0.97
    um = lambda θ : θ 
    uw = lambda θ : θ
    Fm = beta(2,4)
    Fw = beta(2,4)
    λm = 35
    λw = 35

## Agent-Based Simulation

In [ ]:
reset_exog_params()
batches = 10
simulation = []
for v in range(batches):
    men = []
    women = []
    T = 100
    
    for t in range(0,T): 
        # Add new agents 
        for i, θ in enumerate(Fm.rvs(size=λm, random_state=SEED)): 
            men.append(Agent(t*λm + i, t, 'Male', θ, Bm, μ_star))
    
        for i, θ in enumerate(Fw.rvs(size=λw, random_state=SEED)): 
            women.append(Agent(t*λw + i, t, 'Female', θ, Bm, ω_star))  
    
        # Gameplay 
        random.shuffle(men)
        random.shuffle(women) 
        for m, w in zip(men, women): 
            am = m.swipe(w, t)
            aw = w.swipe(m, t)
            m.update(am, aw, t)
            w.update(aw, am, t)
            
        # Logging    
        for a in (men + women): 
            simulation.append(a.info(v, t))  
            
        # Departures
        men_dpts = bernoulli.rvs((1-δ), size=len(men))
        women_dpts = bernoulli.rvs((1-δ), size=len(women))
        
        men[:] = [m for i, m in enumerate(men) if (m.b != 0) and (men_dpts[i] == 0)]
        women[:] = [w for i, w in enumerate(women) if (w.b != 0) and (women_dpts[i] == 0)] 

In [ ]:
data = pd.DataFrame(simulation)
data.head(5)

In [ ]:
total_mass = data.groupby(['batch','time'], as_index=False).size()
fig, ax = plt.subplots()
ax.set_xlim(0,T) 
for i in range(batches):
    ax.plot(range(0,T), total_mass[total_mass.batch==i], color='k')  
ax.set(xlabel=r'Time ($t$)', ylabel=r'Total Agent Mass ($N_t$)')
fig.suptitle('Figure 6: ABM Mass Convergence')
plt.savefig('../../figures/abm-total-conv.png')
plt.show()

In [ ]:
mass_by_sex = data.groupby(['time', 'sex'], as_index=False).size() 
Nm_series = mass_by_sex[mass_by_sex.sex=='Male']['size']
Nw_series = mass_by_sex[mass_by_sex.sex=='Female']['size'] 

In [ ]:
mass_by_sex = data.groupby(['time', 'sex'], as_index=False).size() 
Nm_series = mass_by_sex[mass_by_sex.sex=='Male']['size']
Nw_series = mass_by_sex[mass_by_sex.sex=='Female']['size']

fig, ax = plt.subplots()
ax.set_xlim(0,T) 
ax.plot(range(0,T), Nm_series, color='tab:blue') 
ax.plot(range(0,T), Nw_series, color='tab:pink') 
ax.set(xlabel=r'Time ($t$)', ylabel=r'Agent Mass ($N_t$)')
fig.suptitle('Figure 6: ABM Mass Convergence')
plt.savefig('../../figures/abm-conv.png')
plt.show()
